<a href="https://colab.research.google.com/github/jtsodapop/labelbox-python/blob/master/video_time_export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# P&G Video Time Export
This colab will parse a project's video labels and return the total amount of time in seconds each ontology tool object is part of that label.

Running the code below will result in a locally saved CSV file.

The required fields are the following:
* LABELBOX_API_KEY
* project_id

In [ ]:
!pip install labelbox
!pip install pandas
!pip install numpy
!pip install opencv-python
!pip install requests
!pip install ndjson

In [ ]:
from labelbox import Project, Dataset, DataRow, Client
import pandas as pd
import numpy as np
import cv2
import os
import requests
import ndjson
from google.colab import files

In [ ]:
# you will need to input the API key and project id here
LABELBOX_API_KEY = "API_KEY"
project_id = "PROJECT_ID"

In [ ]:
header = {"Authorization": f"Bearer {LABELBOX_API_KEY}"}

ENDPOINT = "https://api.labelbox.com/graphql"
client = Client(api_key=LABELBOX_API_KEY, endpoint=ENDPOINT)
project = client.get_project(project_id=project_id)

In [ ]:
# generation of the dataframe which will hold our values 
# based on our ontology and additional columns
ontology = project.ontology().normalized
ontology_tools = [tool['name'].lower() for tool in ontology['tools']]

frame_data_df = pd.DataFrame(columns=['created_by','label_id','total_time(s)'])
for tool in ontology_tools:
  frame_data_df[tool] = np.nan

In [ ]:
labels_url = project.export_labels()
labels = requests.get(labels_url).json()

#loop through each label and find the amount of time each annotation is present
for label in labels:
  vidcap = cv2.VideoCapture(label['Labeled Data'])
  fps = vidcap.get(cv2.CAP_PROP_FPS)
  frame_count = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
  vidcap.release()

  #in case there are un-saved or skipped labels
  annotations_url = label['Label'].get('frames', None)
  if annotations_url:
    annotations = ndjson.loads(requests.get(annotations_url, headers=header).text)
    annotations = {
        annotation["frameNumber"]: annotation
        for annotation in annotations
    }

    in_frame = {}
    for frame, annotation in annotations.items():
        for obj in annotation['objects']:
            if obj['value'].lower() in in_frame:
                in_frame[obj['value'].lower()] += 1 / fps
            else:
                in_frame.update({obj['value'].lower(): 1 / fps})

    in_frame['created_by'] = label['Created By']
    in_frame['label_id'] = label['ID']
    in_frame['total_time(s)'] = frame_count / fps

    frame_data_df = frame_data_df.append(in_frame, ignore_index=True)

In [ ]:
frame_data_df.fillna(0, inplace=True)

#create downloadable csv file with project id
frame_data_df.to_csv(f'project_{project_id}_labels.csv', index=False)
files.download(f'project_{project_id}_labels.csv')